In [77]:
import pandas as pd

dija = pd.read_csv('/content/drive/MyDrive/datasets/DJIA_table.csv')
reddit = pd.read_csv('/content/drive/MyDrive/datasets/RedditNews.csv')

dija

,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234
2,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688
3,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703
4,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234
...,...,...,...,...,...,...,...
1984,2008-08-14,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688
1985,2008-08-13,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961
1986,2008-08-12,11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727
1987,2008-08-11,11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609


In [78]:
df = pd.merge(dija, reddit, on='Date')
df.drop('Open', axis=1, inplace=True)
df.drop('High', axis=1, inplace=True)
df.drop('Low', axis=1, inplace=True)
df.drop('Adj Close', axis=1, inplace=True)
df.drop('Volume', axis=1, inplace=True)
df

,Date,Close,News
0,2016-07-01,17949.369141,A 117-year-old woman in Mexico City finally re...
1,2016-07-01,17949.369141,IMF chief backs Athens as permanent Olympic host
2,2016-07-01,17949.369141,"The president of France says if Brexit won, so..."
3,2016-07-01,17949.369141,British Man Who Must Give Police 24 Hours' Not...
4,2016-07-01,17949.369141,100+ Nobel laureates urge Greenpeace to stop o...
...,...,...,...
49713,2008-08-08,11734.320312,b'Why the Pentagon Thinks Attacking Iran is a ...
49714,2008-08-08,11734.320312,b'Caucasus in crisis: Georgia invades South Os...
49715,2008-08-08,11734.320312,b'Indian shoe manufactory - And again in a se...
49716,2008-08-08,11734.320312,b'Visitors Suffering from Mental Illnesses Ban...


In [79]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

X = df['News']
Y = df['Close']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

In [80]:
max_tokens = 64

tokenizer = Tokenizer(num_words=max_tokens)
tokenizer.fit_on_texts(X_train.values)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [81]:
max_len=16

X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)

In [82]:
X_train.shape, X_test.shape

((39774, 16), (9944, 16))

In [83]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)
# Y_train = scaler.fit_transform(Y_train.values.reshape(-1, 1))

In [84]:
X_train.shape, X_test.shape

((39774, 16), (9944, 16))

In [85]:
Y_train.shape

(39774,)

In [86]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [89]:
from keras.models import Sequential
import keras
from keras.layers import LSTM, Dense, Dropout
from keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, LSTM, Flatten

model = Sequential()
model.add(LSTM(100,  activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(LSTM(50,  activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss=keras.losses.MeanSquaredError() , optimizer=keras.optimizers.Adam(), metrics=[keras.metrics.MeanSquaredError(), keras.metrics.MeanAbsoluteError() ])

In [91]:
history = model.fit(X_train, Y_train, batch_size=16, validation_split=0.2, epochs=10, shuffle=False)

Epoch 1/10
1989/1989 [==============================] - 39s 20ms/step - loss: 174763.8906 - mean_squared_error: 174763.8906 - mean_absolute_error: 244.8404 - val_loss: 1503752.7500 - val_mean_squared_error: 1503752.7500 - val_mean_absolute_error: 1015.7019
Epoch 2/10
1989/1989 [==============================] - 43s 22ms/step - loss: 140834.1562 - mean_squared_error: 140834.1562 - mean_absolute_error: 219.7784 - val_loss: 1339726.1250 - val_mean_squared_error: 1339726.1250 - val_mean_absolute_error: 960.7789
Epoch 3/10
1989/1989 [==============================] - 39s 19ms/step - loss: 135157.2812 - mean_squared_error: 135157.2812 - mean_absolute_error: 205.9005 - val_loss: 1421795.5000 - val_mean_squared_error: 1421795.5000 - val_mean_absolute_error: 988.0941
Epoch 4/10
1989/1989 [==============================] - 39s 20ms/step - loss: 133649.2188 - mean_squared_error: 133649.2188 - mean_absolute_error: 201.4229 - val_loss: 1507005.5000 - val_mean_squared_error: 1507005.5000 - val_mean_

In [92]:
preds = model.predict(X_test)

311/311 [==============================] - 5s 13ms/step


In [93]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(Y_test, preds)
print(f'Mean Squared Error: {mse}')

mae = mean_absolute_error(Y_test, preds)
print(f'Mean Absolute Error: {mae}')

Mean Squared Error: 8867347.958912866
Mean Absolute Error: 2739.694111648978
